# Initial Data Load

In [1]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
import pandas as pd

#Had trouble using the SageMaker role to connect to S3 and other services
sess = sagemaker.Session()
role = get_execution_role()

bucket = 'phdata-nlp-s3-bucket'
prefix = 'comprehend-input'
region_name = boto3.Session().region_name

#Read in CSV into dataframe
train_df = pd.read_csv("data/train.csv", names=["Label", "Title", "Review"])

#Extract negative reviews and combine the title and review text into one column
neg_df = train_df[train_df["Label"] == 1]
neg_df["title_and_review"] = neg_df["Title"].astype(str) + " " + neg_df["Review"].astype(str)
neg_df = neg_df.filter(items = ["title_and_review"])

#Grab random sample of data. For reproducability, a seed would be set here
neg_df_samp = neg_df.sample(frac = 0.2)

#Write resulting sample to a csv
neg_df_samp.to_csv("data/sample_neg_reviews.csv")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
#Copy over to S3 bucket. (Had trouble programmatically doing so from Notebook)
!aws s3 cp data/sample_neg_reviews.csv s3://phdata-nlp-s3-bucket/comprehend-input/

#Input the data to Comprehend to receive output matrices
!aws comprehend start-topics-detection-job --number-of-topics 5 --job-name "phdata-comprehend-job" --region region_name --cli-input-json comp.json